In [1]:
!pip install datasets -q -U

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2021.4.0 requires fsspec==2021.04.0, but you have fsspec 2021.10.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import pandas as pd

In [7]:
dataset_name = 'turkish_product_reviews'
dataset = load_dataset(dataset_name)

In [9]:
data = pd.DataFrame(dataset['train'])

In [10]:
len(data)

235165

In [11]:
data['sentiment'].sum()

220284

In [71]:
data.sample(10)

,sentence,sentiment
879,gercekten mükemmel bir ürün surekli yorumlari ...,1
612,"pişirmesi güzel ancak, etlerde kullanılan tuz ...",0
45,tek kelime ile mükemmellll...mutlaka okunması ...,1
656,kalite ve güven teşekkürler hepsiburadaya ve h...,1
1441,"ürün yeni elime ulaştı kaliteli ,çok memnun ka...",1
695,düzceden sipariş ettim 24 saatte elime geçti t...,1
385,gayet iyi fiyatını hakedıyor hepsıburada gayet...,1
866,piyasaya göre fiyatı gerçekten çok uygun telef...,1
1238,ürün 2 günde sorunsuz elime geçti . montaj kol...,1
1160,oldukça etkileyici kesinlikle okunmalı,1


In [13]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20210902T135971
sagemaker bucket: sagemaker-us-east-1-905847418383
sagemaker session region: us-east-1


In [15]:
tokenizer_name = 'dbmdz/bert-base-turkish-uncased'
s3_prefix = 'samples/datasets/turkish_product_reviews'

In [48]:
sample = dataset['train'].train_test_split(test_size=0.1)

Loading cached split indices for dataset at /home/ec2-user/.cache/huggingface/datasets/turkish_product_reviews/default/1.0.0/9cc21a14e05e4117ea24b5b916effe55cbc88278441c21e03d3807c9bda2219d/cache-e4d14b3227264396.arrow and /home/ec2-user/.cache/huggingface/datasets/turkish_product_reviews/default/1.0.0/9cc21a14e05e4117ea24b5b916effe55cbc88278441c21e03d3807c9bda2219d/cache-0b1feb6cc5ad0cca.arrow


In [49]:
sample

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment'],
        num_rows: 211648
    })
    test: Dataset({
        features: ['sentence', 'sentiment'],
        num_rows: 23517
    })
})

In [50]:
dataset2 = sample['test']

In [51]:
train_test = dataset2.train_test_split(test_size=0.1)

In [52]:
train_test

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment'],
        num_rows: 21165
    })
    test: Dataset({
        features: ['sentence', 'sentiment'],
        num_rows: 2352
    })
})

In [53]:
data = pd.DataFrame(train_test['test'])

In [54]:
len(data)

2352

In [55]:
data['sentiment'].sum()

2195

In [56]:
train_dataset = train_test['train']
test_dataset = train_test['test']

In [57]:
train_dataset

Dataset({
    features: ['sentence', 'sentiment'],
    num_rows: 21165
})

In [33]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     |████████████████████████████████| 2.9 MB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 49.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 63.2 MB/s eta 0:00:01


In [34]:
from transformers import AutoTokenizer

In [37]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['sentence'], padding='max_length', truncation=True)

In [58]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [59]:
# set format for pytorch
train_dataset =  train_dataset.rename_column("sentiment", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("sentiment", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [60]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path,fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path,fs=s3)

In [62]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 8,
                 'model_name': tokenizer_name
                 }

In [63]:
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            role=role,
                            transformers_version='4.6',
                            pytorch_version='1.7',
                            py_version='py36',
                            hyperparameters = hyperparameters)

In [64]:
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

2021-10-01 16:52:39 Starting - Starting the training job...
2021-10-01 16:53:05 Starting - Launching requested ML instancesProfilerReport-1633107159: InProgress
.........
2021-10-01 16:54:27 Starting - Preparing the instances for training.........
2021-10-01 16:56:07 Downloading - Downloading input data...
2021-10-01 16:56:27 Training - Downloading the training image...............
2021-10-01 16:59:08 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-01 16:59:07,687 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-01 16:59:07,714 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-01 16:59:07,722 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-01 16:59:08,252 sagemaker-training-toolkit INFO     Invoking user script

Trai

In [65]:
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

-------------------!

In [69]:
sentiment_input= {"inputs": "bu film biraz sıkıcıydı"}

predictor.predict(sentiment_input)

[{'label': 'LABEL_0', 'score': 0.9593140482902527}]